take in the already tagged files and then:

- normalise dates
- combine the files 
- print them out in order

In [15]:
import glob
import csv

In [92]:
def get_tsv_files():
    tsv_files = glob.glob("/Users/ianm/Dropbox/workbench/monthly-tracker/*.txt")
    return tsv_files

In [93]:
def read_tsv_file(filename):
    tsv_data = []
    with open(filename) as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        next(tsvin)
        for row in tsvin:
            date = row[0]
            description = row[1]
            amount = row[2]
            tag = row[3]
            if date != '': tsv_data.append((date, description, amount, tag))
    return tsv_data

In [94]:
tsv_files = get_tsv_files()

In [95]:
def replace_slash_with_hyphen(date):
    date_unslashed = date.replace("/","-")
    return date_unslashed

In [96]:
def separate_year(date):
    date_parts = date.split("-")#remove(str(year))
    for part in date_parts: 
        if len(part) == 4: 
            year = part
            date_parts.remove(part)
    return year, date_parts

In [97]:
def infer_start_end_month(datarows):
    print(len(datarows))
    for date in dates:
        unslashed_date = replace_slash_with_hyphen(date)
        year, date_parts = separate_year(unslashed_date)
        for part in date_parts: 
            if part == "28": # we know the other part of the date is the lower bound month! 
                date_parts.remove("28")
                month_one = int(date_parts[0])
                if month_one == 12: 
                    month_two = '01'
                else:
                    month_two = str(month_one +1)
                return(month_one, month_two)

In [98]:
def normalise_date(date: str, month_one: str, month_two: str) -> str:
    start_day = 23
    unslashed_date = replace_slash_with_hyphen(date)
    year, date_parts = separate_year(unslashed_date)
    #print(date_parts[-1], month_one)
    if date_parts[-1] == str(month_one):
        if date_parts[0] >= str(start_day):
            date_parts.reverse()
    if date_parts[-1] == month_two:
        date_parts.reverse()
    normalised_date = year + '-' + date_parts[0] + '-' + date_parts[1]
    return normalised_date 

In [101]:
all_tagged_data = []
for tfile in tsv_files:
    try:
        data = read_tsv_file(tfile)
        dates = [x[0] for x in data]
        start, end = infer_start_end_month(dates)
        normalised_dates = [normalise_date(x, start, end) for x in dates]
        normalised_data = []
        for index, row in enumerate(data):
            normalised_data.append((normalised_dates[index],row[1],row[2], row[3]))
        normalised_data.sort()
        all_tagged_data.extend(normalised_data)
    except:
        print(tfile)

128
132
165


In [103]:
all_tagged_data.sort()

In [105]:
len(all_tagged_data)

425

In [106]:
%store all_tagged_data 

Stored 'all_tagged_data' (list)


In [109]:
test_tagged_data = all_tagged_data[0:5]

In [110]:
test_tagged_data

[('2018-10-22', 'PRET A MANGERCITY ROAD LONDON', '6.43', 'lunch'),
 ('2018-10-22', "SAINSBURY'S BROMLEYBY LONDON", '186.37', 'food'),
 ('2018-10-23', 'TFL TRAVEL CHARGE TFL.GOV.UK/CP', '5.8', 'travel'),
 ('2018-10-24', '', '292', 'travel'),
 ('2018-10-24', '', '330', 'travel')]

In [111]:
test_tagged_data_kyes = [(x[0], x[1], x[2]) for x in test_tagged_data]

In [112]:
test_tagged_data_kyes

[('2018-10-22', 'PRET A MANGERCITY ROAD LONDON', '6.43'),
 ('2018-10-22', "SAINSBURY'S BROMLEYBY LONDON", '186.37'),
 ('2018-10-23', 'TFL TRAVEL CHARGE TFL.GOV.UK/CP', '5.8'),
 ('2018-10-24', '', '292'),
 ('2018-10-24', '', '330')]

In [ ]:
for spend in test_tagged_data_kyes:
    if spend in 